In [ ]:
#単純な共起行列を作成
#リーダーボードでscore～0.0681：859位2022/4/18時点
#for文が多いので時間がかかる&メモリを消費する。google colab上では実行可能
#実行時間はpro環境

In [1]:
#colab_note上で必要
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

In [3]:
folder_path = '/content/drive/My Drive/dressipi_recsys2022/'
file_name_list = ["candidate_items.csv", "item_features.csv", "test_final_sessions.csv", "test_leaderboard_sessions.csv", "train_purchases.csv", "train_sessions.csv"]
df_candidate_items = pd.read_csv(folder_path + file_name_list[0])
df_item_features = pd.read_csv(folder_path + file_name_list[1])
df_test_final_sessions = pd.read_csv(folder_path + file_name_list[2])
df_test_leaderboard_sessions = pd.read_csv(folder_path + file_name_list[3])
df_train_purchases = pd.read_csv(folder_path + file_name_list[4])
df_train_sessions = pd.read_csv(folder_path + file_name_list[5])

In [6]:
#item_idと共起行列上のindexを変換するdictを作成
unique_item_id = df_item_features.sort_values('item_id')['item_id'].unique()
item_dict = {unique_item_id[i]: i for i in range(0, len(unique_item_id))}
item_dict_inv = {i:unique_item_id[i] for i in range(0, len(unique_item_id))}

In [5]:
#各セッションの、閲覧アイテムと購入アイテムをリスト化
train_session_item_id = df_train_sessions.groupby('session_id')['item_id'].unique()
train_purchase_item_id = df_train_purchases.groupby('session_id')['item_id'].unique()

In [7]:
#共起行列的な距離(item_dict)を定義　20秒くらい
item_dist = np.zeros([len(unique_item_id), len(unique_item_id)])

session_id_list = df_train_purchases['session_id'].unique()
for s in session_id_list:
  purchased_item = train_purchase_item_id[s][0]
  temp_list = train_session_item_id[s]
#  print(purchased_item)
#  print(temp_list)
  for t in temp_list:
    item_dist[item_dict[t]][item_dict[purchased_item]] += 1

In [8]:
#規格化している　30秒くらい
for i in range(len(item_dist)):
  item_dist[::,i] = item_dist[::,i]/np.sum(item_dist[::,i])
item_dist = np.nan_to_num(item_dist,0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
#candidateにないものは雑にdist = 0 で処理する
unique_item_id_candidate = df_candidate_items.sort_values('item_id')['item_id'].unique()
remove_item_id = unique_item_id[np.isin(unique_item_id, unique_item_id_candidate) == False]
remove_item_index = [item_dict[item_id] for item_id in remove_item_id]
item_dist[::,remove_item_index] = 0.0

In [10]:
#リーダーボードのセッションリストを作成
unique_session_lb = df_test_leaderboard_sessions['session_id'].unique()
session_items_lb = df_test_leaderboard_sessions.groupby('session_id')['item_id'].unique()

In [12]:
#予測：雑実装のため5分くらいかかる、、、

df_sub_lb = pd.DataFrame(columns=['session_id', 'item_id', 'rank'])
item_id = []
session_id = []
count = 0

for s in unique_session_lb:
  #for文のカウント,50000で終わり
  if(count%10000 == 0):
    print(count)
  count += 1

　#予測パート
  temp_session = session_items_lb[s]
  temp_prob = [item_dict[i] for i in temp_session]
  temp_prob = np.sum(item_dist[temp_prob], axis = 0) #
  prob_top100_index = np.argsort(-temp_prob)[0:100]
  prob_top100_item_id = np.array([item_dict_inv[i] for i in prob_top100_index])
 
　#結果の代入
  item_id = np.concatenate([item_id,prob_top100_item_id])
  temp = np.array([s for i in range(100)])
  session_id = np.concatenate([session_id,temp])

0
10000
20000
30000
40000


In [14]:
#最終データ作成
rank = np.tile(np.array([i + 1 for i in range(100)]),len(unique_session_lb))
df_sub_lb = pd.DataFrame(np.vstack([session_id, item_id, rank]).T, columns=['session_id', 'item_id', 'rank'])
df_sub_lb = df_sub_lb.astype('int')#intに明示的に変換しないと通らない

In [15]:
file_name = "submit2022041718"
df_sub_lb.to_csv(folder_path + file_name + ".csv", index=False)